In [24]:
import numpy  as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import keras
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
%matplotlib inline
from catboost import CatBoostClassifier, FeaturesData
import datetime as DT
import io

Importando arquivos

In [6]:
tabelao = pd.read_csv('tabelao_idade.csv')

In [7]:
tabelao.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 96406 entries, 0 to 96405
Data columns (total 21 columns):
CODIGO_CLIENTE             96406 non-null int64
DATA_NASCIMENTO            96403 non-null object
IDADE                      96406 non-null int64
SEXO                       96405 non-null object
ESTADO_CIVIL               96405 non-null object
CIDADE                     96406 non-null object
UF                         96406 non-null object
BAIRRO                     96406 non-null object
CEP                        96406 non-null int64
QTD_DEPENDENTES            96405 non-null float64
DATA_CADASTRO              96406 non-null object
DATA_ADMISSAO              52050 non-null object
CATEGORIAL_PROFISSAO       94690 non-null object
TIPO_RESIDENCIA            95454 non-null object
RENDA_TITULAR              96400 non-null float64
RENDA_CJ                   4393 non-null float64
OUTRAS_RENDAS              11070 non-null float64
QTD_CARTOES_ADICIONAIS     96406 non-null int64
FLAG_CONTA

In [8]:
tabelao = tabelao.drop(columns=['CODIGO_CLIENTE', 'DATA_NASCIMENTO', 'UF',
             'BAIRRO', 'CEP', 'DATA_CADASTRO','DATA_ADMISSAO',
             'RENDA_CJ','OUTRAS_RENDAS','FLAG_CONTA_BANCO',
                                'QTD_CARTOES_ADICIONAIS','VALOR_HISTORICO_COMPRAS'], axis=1)

In [9]:
tabelao.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 96406 entries, 0 to 96405
Data columns (total 9 columns):
IDADE                   96406 non-null int64
SEXO                    96405 non-null object
ESTADO_CIVIL            96405 non-null object
CIDADE                  96406 non-null object
QTD_DEPENDENTES         96405 non-null float64
CATEGORIAL_PROFISSAO    94690 non-null object
TIPO_RESIDENCIA         95454 non-null object
RENDA_TITULAR           96400 non-null float64
CLASS                   96406 non-null int64
dtypes: float64(2), int64(2), object(5)
memory usage: 6.6+ MB


In [10]:
tabelao.head()

,IDADE,SEXO,ESTADO_CIVIL,CIDADE,QTD_DEPENDENTES,CATEGORIAL_PROFISSAO,TIPO_RESIDENCIA,RENDA_TITULAR,CLASS
0,63,M,SO,BENTO GONCALVES,0.0,APOSENTADO,PR,2300.0,1
1,51,M,CA,PELOTAS,2.0,ASSALARIADO,OU,1500.0,1
2,30,M,SO,ALEGRETE,0.0,AUT+NOMO,PR,1000.0,1
3,65,F,SO,GARIBALDI,0.0,APOSENTADO,OU,1050.0,1
4,62,F,SO,TRES PASSOS,0.0,AUT+NOMO,PR,1200.0,1


In [19]:
tabelao.CLASS.value_counts()

1    72127
0    24279
Name: CLASS, dtype: int64

In [20]:
X = tabelao.iloc[:, :-1].values

In [21]:
X

array([[63, 'M', 'SO', ..., 'APOSENTADO', 'PR', 2300.0],
       [51, 'M', 'CA', ..., 'ASSALARIADO', 'OU', 1500.0],
       [30, 'M', 'SO', ..., 'AUT+NOMO', 'PR', 1000.0],
       ...,
       [39, 'M', 'SO', ..., nan, 'PR', 2000.0],
       [59, 'F', 'CA', ..., 'ASSALARIADO', 'PR', 3264.27],
       [37, 'F', 'CO', ..., 'ASSALARIADO', 'PR', 1347.99]], dtype=object)

In [22]:
y = tabelao.iloc[:, -1].values

In [23]:
y

array([1, 1, 1, ..., 1, 0, 1])

In [25]:
skf = StratifiedKFold(n_splits=5,)

In [ ]:
for train_index, test_index in skf.split(X, y):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    